In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px

sber_data = pd.read_csv('data_2/sber_data.csv', sep=',')
#display(sber_data.head())
#Чему равно число строк в таблице?
#display(sber_data.tail())

#Сколько районов Москвы и Московской области представлено в данных?
#display(sber_data['sub_area'].value_counts())

#Чему равна максимальная цена квартир (price_doc)? Введите это число полностью, без округлений.
#display(sber_data['price_doc'].max())


#Проверим, влияет ли уровень экологической обстановки в районе на цену квартиры.
# Постройте коробчатую диаграмму цен на квартиры (price_doc) в зависимости от уровня экологической обстановки в районе (ecology).
# Какой уровень ценится на рынке меньше всего?
#Строим график распределения возраста в зависимости от лояльности
fig = px.box(
    sber_data,
    x='price_doc',
    y='ecology',
    color='ecology',
    orientation = 'h',
    labels={'price_doc':'Цены на квартиры', 'ecology':'Уровень экологической обстановки'},
    height = 800, 
    width=1200,
    title = "Распределение цен на квартиры взависимости от уровеня экологической обстановки",
)
#fig.show()


#Постройте диаграмму рассеяния, которая покажет, как цена на квартиру (price_doc) связана с расстоянием до центра Москвы (kremlin_km). 
#Выберите все верные утверждения.
fig = px.scatter(
    sber_data,
    x='kremlin_km',
    y='price_doc',
    labels={'price_doc':'Цены на квартиры, млн.', 'kremlin_km':'Hасстоянием до центра Москвы, км.'},
    height = 800, 
    width=1200,
    title = "Lиаграмму рассеяния зависимости цены на квартиру и расстояния до центра Москвы",
)
#fig.show()



#В библиотеке pandas специально для этого реализован метод isnull().
# Этот метод возвращает новый DataFrame, в ячейках которого стоят булевы значения True и False.
# True ставится на месте, где ранее находилось значение NaN.
#display(sber_data.isnull().tail())


#Первый способ — это вывести на экран названия столбцов, где число пропусков больше 0/
#Для этого вычислим средний по столбцам результат метода isnull(). Получим долю пропусков в каждом столбце.
#Умножаем на 100 %, находим столбцы, где доля пропусков больше 0, сортируем по убыванию и выводим результат:
cols_null_percent = sber_data.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
#display(cols_with_null)


#Можно воспользоваться столбчатой диаграммой, чтобы визуально оценить соотношение числа пропусков к числу записей.
# Самый быстрый способ построить её — использовать метод plot():
#cols_with_null.plot(
#    kind='bar',
#    figsize=(10, 4),
#    title='Распределение пропусков в данных'
#);



#Для создания такой тепловой карты можно воспользоваться результатом метода isnull().
# Ячейки таблицы, в которых есть пропуск, будем отмечать жёлтым цветом, а остальные — синим.
# Для этого создадим собственную палитру цветов тепловой карты с помощью метода color_pallete() из библиотеки seaborn.
#colors = ['blue', 'yellow'] 
#fig = plt.figure(figsize=(10, 4))
#cols = cols_with_null.index
#ax = sns.heatmap(
#    sber_data[cols].isnull(),
#    cmap=sns.color_palette(colors),
#)


#редварительно создадим копию исходной таблицы — drop_data, чтобы не повредить её.
#Зададимся порогом в 70 %: будем оставлять только те столбцы, в которых 70 и более процентов записей не являются пустыми.
#После этого удалим записи, в которых содержится хотя бы один пропуск.
#Наконец, выведем информацию о числе пропусков и наслаждаемся нулями. 
#создаем копию исходной таблицы
drop_data = sber_data.copy()
#задаем минимальный порог: вычисляем 70% от числа строк
thresh = drop_data.shape[0]*0.7
#удаляем столбцы, в которых более 30% (100-70) пропусков
drop_data = drop_data.dropna(thresh=thresh, axis=1)
#удаляем записи, в которых есть хотя бы 1 пропуск
drop_data = drop_data.dropna(how='any', axis=0)
#отображаем результирующую долю пропусков
drop_data.isnull().mean()

#print(drop_data.shape)



#Вся сложность заключается в выборе метода заполнения.
# Важным фактором при выборе метода является распределение признаков с пропусками.
# Давайте выведем их на экран. 
# pandas это можно сделать с помощью метода hist():
#cols = cols_with_null.index
#sber_data[cols].hist(figsize=(20, 8));


#Заполнение значений осуществляется с помощью метода fillna().
# Главный параметр метода — value (значение, на которое происходит заполнение данных в столбце).
# Если метод вызывается от имени всего DataFrame, то в качестве value можно использовать словарь,
# где ключи — названия столбцов таблицы, а значения словаря — заполняющие константы. 
#Создадим такой словарь, соблюдая рекомендации, приведённые выше, а также копию исходной таблицы.
# Произведём операцию заполнения с помощью метода fillna() и удостоверимся, что пропусков в данных больше нет:
#создаем копию исходной таблицы
fill_data = sber_data.copy()
#создаем словарь имя столбца: число(признак) на который надо заменить пропуски
values = {
    'life_sq': fill_data['full_sq'],
    'metro_min_walk': fill_data['metro_min_walk'].median(),
    'metro_km_walk': fill_data['metro_km_walk'].median(),
    'railroad_station_walk_km': fill_data['railroad_station_walk_km'].median(),
    'railroad_station_walk_min': fill_data['railroad_station_walk_min'].median(),
    'hospital_beds_raion': fill_data['hospital_beds_raion'].mode()[0],
    'preschool_quota': fill_data['preschool_quota'].mode()[0],
    'school_quota': fill_data['school_quota'].mode()[0],
    'floor': fill_data['floor'].mode()[0]
}
#заполняем пропуски в соответствии с заявленным словарем
fill_data = fill_data.fillna(values)
#выводим результирующую долю пропусков
fill_data.isnull().mean()


#Посмотрим, на то, как изменились распределения наших признаков:
#cols = cols_with_null.index
#fill_data[cols].hist(figsize=(20, 8));
#Обратите внимание на то, как сильно изменилось распределение для признака hospital_beds_raion.
# Это связано с тем, что мы заполнили модальным значением почти 47 % общих данных.
# В результате мы кардинально исказили исходное распределение признака, что может плохо сказаться на модели.




#Заполнение недостающих значений константами с добавлением индикатора
# Посмотрим на реализацию. Как обычно, создадим копию indicator_data исходной таблицы.
# В цикле пройдёмся по столбцам с пропусками и будем добавлять в таблицу новый признак (с припиской "was_null"), который получается из исходного с помощью применения метода isnull().
# После чего произведём обычное заполнение пропусков, которое мы совершали ранее, и выведем на экран число отсутствующих значений в столбце, чтобы убедиться в результате:
#создаем копию исходной таблицы
indicator_data = sber_data.copy()
#в цикле пробегаемся по названиям столбцов с пропусками
for col in cols_with_null.index:
    #создаем новый признак-индикатор как col_was_null
    indicator_data[col + '_was_null'] = indicator_data[col].isnull()
#создаем словарь имя столбца: число(признак) на который надо заменить пропуски   
values = {
    'life_sq': indicator_data['full_sq'],
    'metro_min_walk': indicator_data['metro_min_walk'].median(),
    'metro_km_walk': indicator_data['metro_km_walk'].median(),
    'railroad_station_walk_km': indicator_data['railroad_station_walk_km'].median(),
    'railroad_station_walk_min': indicator_data['railroad_station_walk_min'].median(),
    'hospital_beds_raion': indicator_data['hospital_beds_raion'].mode()[0],
    'preschool_quota': indicator_data['preschool_quota'].mode()[0],
    'school_quota': indicator_data['school_quota'].mode()[0],
    'floor': indicator_data['floor'].mode()[0]
}
#заполняем пропуски в соответствии с заявленным словарем
indicator_data = indicator_data.fillna(values)
#выводим результирующую долю пропусков
#indicator_data.isnull().mean()
#display(indicator_data.head())
#Метод исходит из предположения, что, если дать модели информацию о том, что в ячейке ранее была пустота, 
# то она будет меньше доверять таким записям и меньше учитывать её в процессе обучения. Иногда такие фишки действительно работают, 
# иногда не дают эффекта, а иногда и вовсе могут ухудшить результат обучения и затруднить процесс обучения.


#Комбинирование методов

#Наверняка вы уже догадались, что необязательно использовать один метод. Вы можете их комбинировать. Например, мы можем:
#удалить столбцы, в которых более 30 % пропусков;
#удалить записи, в которых более двух пропусков одновременно;
#заполнить оставшиеся ячейки константами.
#Посмотрим на реализацию такого подхода в коде:
#создаём копию исходной таблицы
combine_data = sber_data.copy()

#отбрасываем столбцы с числом пропусков более 30% (100-70)
n = combine_data.shape[0] #число строк в таблице
thresh = n*0.7
combine_data = combine_data.dropna(thresh=thresh, axis=1)

#отбрасываем строки с числом пропусков более 2 в строке
m = combine_data.shape[1] #число признаков после удаления столбцов
combine_data = combine_data.dropna(thresh=m-2, axis=0)

#создаём словарь 'имя_столбца': число (признак), на который надо заменить пропуски 
values = {
    'life_sq': combine_data['full_sq'],
    'metro_min_walk': combine_data['metro_min_walk'].median(),
    'metro_km_walk': combine_data['metro_km_walk'].median(),
    'railroad_station_walk_km': combine_data['railroad_station_walk_km'].median(),
    'railroad_station_walk_min': combine_data['railroad_station_walk_min'].median(),
    'preschool_quota': combine_data['preschool_quota'].mode()[0],
    'school_quota': combine_data['school_quota'].mode()[0],
    'floor': combine_data['floor'].mode()[0]
}
#заполняем оставшиеся записи константами в соответствии со словарем values
combine_data = combine_data.fillna(values)
#выводим результирующую долю пропусков
#display(combine_data.isnull().mean())

#print(combine_data.shape)

In [22]:
#Пусть у нас есть признак, по которому мы будем искать выбросы.
# Давайте рассчитаем его статистические показатели (минимум, максимум, среднее, квантили)
# и по ним попробуем определить наличие аномалий.
sber_data['life_sq'].describe()

#Найдём число квартир с нулевой жилой площадью:
#print(sber_data[sber_data['life_sq'] == 0].shape[0])

#А теперь выведем здания с жилой площадью более 7 000 квадратных метров:
#display(sber_data[sber_data['life_sq'] > 7000])


#Логичен вопрос: а много ли у нас таких квартир, у которых жилая площадь больше, чем суммарная?
outliers = sber_data[sber_data['life_sq'] > sber_data['full_sq']]
#print(outliers.shape[0])


#Таких квартир оказывается 37 штук. Подобные наблюдения уже не поддаются здравому смыслу — они являются ошибочными, 
# и от них стоит избавиться. Для этого можно воспользоваться методом drop() и удалить записи по их индексам:
cleaned = sber_data.drop(outliers.index, axis=0)
#print(f'Результирующее число записей: {cleaned.shape[0]}')

#Ещё пример: давайте посмотрим на признак числа этажей (floor).
#display(sber_data['floor'].describe())

#Снова видим подозрительную максимальную отметку в 77 этажей. Проверим все квартиры, которые находятся выше 50 этажей:
#display(sber_data[sber_data['floor']> 50])
#Всего одна квартира в Ломоносовском районе. Пора идти в интернет в поиске самых высоких зданий в Москве! 

#На гистограмме мы можем увидеть потенциальные выбросы как низкие далеко отстоящие от основной группы столбцов «пеньки», 
# а на коробчатой диаграмме — точки за пределами усов

#Построим гистограмму и коробчатую диаграмму для признака полной площади (full_sq):
#fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
#histplot = sns.histplot(data=sber_data, x='full_sq', ax=axes[0]);
#histplot.set_title('Full Square Distribution');
#boxplot = sns.boxplot(data=sber_data, x='full_sq', ax=axes[1]);
#boxplot.set_title('Full Square Boxplot');



#В соответствии с этим алгоритмом напишем функцию outliers_iqr(), которая вам может ещё не раз пригодиться в реальных задачах.
# Эта функция принимает на вход DataFrame и признак, по которому ищутся выбросы, а затем возвращает потенциальные выбросы,
# найденные с помощью метода Тьюки, и очищенный от них датасет.

#Квантили вычисляются с помощью метода quantile().
# Потенциальные выбросы определяются при помощи фильтрации данных по условию выхода за пределы верхней или нижней границы.

#Одним из таких подходов является метод межквартильного размаха (его еще называют методом Тьюки), 
# который используется для построения коробчатой диаграммы.

#вычислить 25-ый и 75-ый квантили (первый и третий квартили) — и  для признака, который мы исследуем;

def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

#Применим эту функцию к таблице sber_data и признаку full_sq, а также выведем размерности результатов:
outliers, cleaned = outliers_iqr(sber_data, 'full_sq')
#print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
#print(f'Результирующее число записей: {cleaned.shape[0]}')
#display(outliers.head())
#display(cleaned.head())


#Согласно классическому методу Тьюки, под выбросы у нас попали 963 записи в таблице.
# Давайте построим гистограмму и коробчатую диаграмму на новых данных cleaned_sber_data:

#fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
#histplot = sns.histplot(data=cleaned, x='full_sq', ax=axes[0]);
#histplot.set_title('Cleaned Full Square Distribution');
#boxplot = sns.boxplot(data=cleaned, x='full_sq', ax=axes[1]);
#boxplot.set_title('Cleaned Full Square Boxplot');


#Построим две гистограммы признака расстояния до МКАД (mkad_km): первая — в обычном масштабе, а вторая — в логарифмическом.
# Логарифмировать будем с помощью функции log() из библиотеки numpy (натуральный логарифм — логарифм по основанию числа e).
# Признак имеет среди своих значений 0. Из математики известно, что логарифма от 0 не существует, поэтому мы прибавляем к
# нашему признаку 1, чтобы не логарифмировать нули и не получать предупреждения.
#fig, axes = plt.subplots(1, 2, figsize=(15, 4))

#гистограмма исходного признака
#histplot = sns.histplot(sber_data['mkad_km'], bins=30, ax=axes[0])
#histplot.set_title('MKAD Km Distribution');

#гистограмма в логарифмическом масштабе
#log_mkad_km= np.log(sber_data['mkad_km'] + 1)
#histplot = sns.histplot(log_mkad_km , bins=30, ax=axes[1])
#histplot.set_title('Log MKAD Km Distribution');


#Примечание: Численный показатель асимметрии можно вычислить с помощью метода:
#print(log_mkad_km.skew())
# -0.14263612203024953


#На вход она принимает DataFrame и признак, по которому ищутся выбросы.
# В дополнение добавим в функцию возможность работы в логарифмическом масштабе: для этого введём аргумент log_scale.
# Если он равен True, то будем логарифмировать рассматриваемый признак, иначе — оставляем его в исходном виде.

#Как и раньше, функция будет возвращать выбросы и очищенные от них данные:

def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned


#Применим эту функцию к таблице sber_data и признаку mkad_km, а также выведем размерности результатов:
outliers, cleaned = outliers_z_score(sber_data, 'mkad_km', log_scale=True)
#print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
#print(f'Результирующее число записей: {cleaned.shape[0]}')


#Итак, метод z-отклонения нашел нам 33 потенциальных выброса по признаку расстояния до МКАД.
# Давайте узнаем, в каких районах (sub_area) представлены эти квартиры:
#print(outliers['sub_area'].unique())



#озможно, мы не учли того факта, что наш логарифм распределения всё-таки не идеально нормален и в нём присутствует
# некоторая асимметрия. Возможно, стоит дать некоторое «послабление» на границы интервалов? Давайте отдельно построим
# гистограмму прологарифмированного распределения, а также отобразим на гистограмме вертикальные линии, соответствующие
# среднему (центру интервала в методе трёх сигм) и границы интервала . Вертикальные линии можно построить с помощью метода axvline().
# Для среднего линия будет обычной, а для границ интервала — пунктирной (параметр ls ='--'):
#fig, ax = plt.subplots(1, 1, figsize=(8, 4))
#log_mkad_km = np.log(sber_data['mkad_km'] + 1)
#histplot = sns.histplot(log_mkad_km, bins=30, ax=ax)
#histplot.axvline(log_mkad_km.mean(), color='k', lw=2)
#histplot.axvline(log_mkad_km.mean()+ 3 * log_mkad_km.std(), color='k', ls='--', lw=2)
#histplot.axvline(log_mkad_km.mean()- 3 * log_mkad_km.std(), color='k', ls='--', lw=2)
#histplot.set_title('Log MKAD Km Distribution');



#Итак, мы графически построили интервал метода трёх сигм поверх нашего распределения.
# Он показывает, какие наблюдения мы берём в интервал, а какие считаем выбросами. Легко заметить,
# что среднее значение (жирная вертикальная линия) находится левее моды. Это свойство распределений с левосторонней асимметрией.
# Также видны наблюдения, которые мы не захватили своим интервалом (небольшой «пенёк» правее верхней границы) — это и есть наши
# квартиры из поселений «Роговское» и «Киевский». Очевидно, что если немного (меньше чем на одну сигму) «сдвинуть» верхнюю границу
# вправо, то мы захватим эти наблюдения. Давайте сделаем это.

def outliers_z_score_mod(data, feature, log_scale=False, left=3, right=3):
    if isinstance(data, str):  # Проверка, если передан путь к файлу
        data = pd.read_csv(data)
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned


#outliers, cleaned = outliers_z_score_mod(sber_data, 'mkad_km', log_scale=True, left = 3, right = 3.5)
#print(f'Число выбросов по методу : {outliers.shape[0]}')
#print(f'Результирующее число записей: {cleaned.shape[0]}')


#Постройте гистограмму для признака price_doc в логарифмическом масштабе.
# А также, добавьте на график линии, отображающие среднее и границы интервала для метода трех сигм. Выберите верные утверждения:
#fig, ax = plt.subplots(1, 1, figsize=(8, 4))
#log_price_doc = np.log(sber_data['price_doc'] + 1)
#histplot = sns.histplot(log_price_doc, bins=30, ax=ax)
#histplot.axvline(log_price_doc.mean(), color='k', lw=2)
#histplot.axvline(log_price_doc.mean()+ 3 * log_price_doc.std(), color='k', ls='--', lw=2)
#histplot.axvline(log_price_doc.mean()- 3 * log_price_doc.std(), color='k', ls='--', lw=2)
#histplot.set_title('Log price_doc');



#Найдите потенциальные выбросы по признаку price_doc с помощью метода z-отклонения. Используйте логарифмический масштаб распределения.
# Сделайте «послабление» на 0.7 сигм в обе стороны распределения. Сколько выбросов вы получили?
#outliers, cleaned = outliers_z_score_mod(sber_data, 'price_doc', log_scale=True, left = 3.7, right = 3.7)
#print(f'Число выбросов по методу : {outliers.shape[0]}')
#print(f'Результирующее число записей: {cleaned.shape[0]}')



#Добавьте фишку с логарифмированием в свою функцию outliers_iqr_mod(). Добавьте в неё параметр log_scale.
# Если он выставлен в True, то производится логарифмирование признака. Примените полученную функцию к признаку price_doc.
# Число межквартильных размахов в обе стороны обозначьте как 3. Чему равно число выбросов, полученных таким методом?
def outliers_iqr_mod_log(data, feature, log_scale=False, left=1.5, right=1.5):
    #if isinstance(data, str):  # Проверка, если передан путь к файлу
     #   data = pd.read_csv(data)
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]

    
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75)
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)

    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

#outliers, cleaned = outliers_iqr_mod_log(sber_data, 'price_doc', log_scale=True, left=3, right=3)
#print(f'Число выбросов по методу Тьюки + log: {outliers.shape[0]}')
#print(f'Результирующее число записей: {cleaned.shape[0]}')


#Обнаружение и ликвидация дубликатов

#Проверим, есть у нас такие записи: для этого сравним число уникальных значений в столбце id с числом строк.
# Число уникальных значений вычислим с помощью метода nunique():
sber_data['id'].nunique() == sber_data.shape[0]


#Чтобы отследить дубликаты, можно воспользоваться методом duplicated(), который возвращает булеву маску для фильтрации.
# Для записей, у которых совпадают признаки, переданные методу, он возвращает True, для остальных — False.

#У метода есть параметр subset — список признаков, по которым производится поиск дубликатов.
# По умолчанию используются все столбцы в DataFrame и ищутся полные дубликаты.

#Найдём число полных дубликатов таблице sber_data. Предварительно создадим список столбцов dupl_columns,
# по которым будем искать совпадения (все столбцы, не включая id). 

#Создадим маску дубликатов с помощью метода duplicated() и произведём фильтрацию.
# Результат заносим в переменную sber_duplicates. Выведем число строк в результирующем DataFrame:

dupl_columns = list(sber_data.columns)
dupl_columns.remove('id')

mask = sber_data.duplicated(subset=dupl_columns)
sber_duplicates = sber_data[mask]
#print(f'Число найденных дубликатов: {sber_duplicates.shape[0]}')



#еперь нам необходимо от них избавиться. Для этого легче всего воспользоваться методом drop_duplicates(),
# который удаляет повторяющиеся записи из таблицы. 
#Создадим новую таблицу sber_dedupped, которая будет версией исходной таблицы, очищенной от полных дубликатов.

sber_dedupped = sber_data.drop_duplicates(subset=dupl_columns)
#print(f'Результирующее число записей: {sber_dedupped.shape[0]}')


#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in sber_data.columns:
    #наибольшая относительная частота в признаке
    top_freq = sber_data[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = sber_data[col].nunique() / sber_data[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

#Итак, мы нашли шесть неинформативных признаков. Теперь можно удалить их с помощью метода drop(),
#передав результирующий список в его аргументы.
information_sber_data = sber_data.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {information_sber_data.shape[1]}')

id: 100.0% уникальных значений
oil_chemistry_raion: 99.03% одинаковых значений
railroad_terminal_raion: 96.27% одинаковых значений
nuclear_reactor_raion: 97.17% одинаковых значений
big_road1_1line: 97.44% одинаковых значений
mosque_count_1000: 98.08% одинаковых значений
Результирующее число признаков: 55
